In [1]:
### Preface
import javabridge
import bioformats as bf
javabridge.start_vm(class_path=bf.JARS)
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sys import path as syspath
syspath.append("./functions/")
from physio_def_1 import getApparentFreq, importFrames, getTimes

from plotFirst_1 import plotImage

from shutil import move as movecarefully

import os

In [2]:
from collections import OrderedDict

In [3]:
def do_single_Series(iSeries):
    from warnings import warn
    try:
        im = xml.image(iSeries)
        Name = im.Name

        if Nimages==1:
            npzFile   = filename.replace(extension,".npz")
            imageFile = filename.replace(extension,".png")
            dimFile   = filename.replace(extension,".txt")
        else:
            npzFile   = filename.replace(extension,f"_{iSeries}_{Name}.npz")
            imageFile = filename.replace(extension,f"_{iSeries}_{Name}.png")
            dimFile   = filename.replace(extension,f"_{iSeries}_{Name}.txt")

        npzFile   = os.path.join(newdir,npzFile   )
        imageFile = os.path.join(newdir,imageFile )
        dimFile   = os.path.join(newdir,dimFile   )

        dimensions = dict(zip("TXY",(getattr(im.Pixels, "Size"+dim) for dim in "TXY")))

        if dimensions["T"]>1:
            dimensions['freq'] = getApparentFreq(iSeries,xml)

        howManyFirstFrames = min(10,dimensions["T"])
        firstFrames = importFrames(rdr,idx=iSeries, which=(howManyFirstFrames,))

        extraDims = len(firstFrames.shape)>3
        if extraDims:
        #         print("There are more dimensions here than expected.\nI'll remedy this for now, but be shure to keep this in mind.")
            retainDims = np.where(firstFrames.std(axis=tuple(range(len(firstFrames.shape)-1)))>0)[0]
            assert len(retainDims)==1
            retainDims = int(retainDims[0])
            firstFrames = firstFrames.T[retainDims].T
        else:
            retainDims = None

        meanFirstFrames = firstFrames.mean(axis=0)
        #     stdFirstFrames = firstFrames.std(axis=0)

        pxSize = im.Pixels.get_PhysicalSizeX()
        pxUnit = im.Pixels.get_PhysicalSizeXUnit()
        text = "\n".join([" %s:%i"%(c,dimensions[c])  for c in "XYT"])
        if dimensions["T"]>1:
            text += "\n f:%.1f Hz"%dimensions['freq']
        dimensions["pxSize"] = pxSize
        dimensions["pxUnit"] = pxUnit
        pd.DataFrame([dimensions]).to_csv(dimFile, index=False)
        
        try:
            plotImage(meanFirstFrames.T,pxSize=pxSize,pxUnit=pxUnit,
                      savePath=imageFile,
                      addInfo=text)
            plt.close()
        except:
            pass

        time = getTimes(xml_=xml,idx_=iSeries)

        mintype = pd.to_numeric(firstFrames.flatten(),downcast="unsigned").dtype
        allimage = importFrames(
            rdr,
            idx=iSeries,
            dtype=mintype,
            which=(dimensions['T'],None,None,retainDims)[:3+int(extraDims)]
        #     which=(None,None,None)+tuple([retainDims][:int(extraDims)])
        )
        np.savez_compressed(npzFile,time=time,data=allimage)
        return 0
    except:
        warn(f"something was off doing {iSeries} in file {pathToRecord}.\nI'll continue, but with a note in the folder.")
        with open(os.path.join(newdir,filename.replace(extension,".err")),"a") as fw:
            fw.write(f"error for series number {iSeries}\n")
        return 1

In [4]:
def prepareOverview(directory):    
    overview = []
    for txtname in os.listdir(directory)[::-1]:
        if not txtname.endswith("txt"):
            continue
        try:
            sr = pd.read_csv(os.path.join(directory,txtname)).loc[0]
            sr = sr.append(pd.Series({
                "filename"    :    txtname.rstrip(".txt"),
                "iSeries"     :int(txtname.split("_")[-2]),
                "Series_Name" :    txtname.split("_")[-1],
            }))
        except:
            sr = pd.Series({"filename":txtname})
        overview += [sr]

    overview = pd.DataFrame(overview)
    overview.set_index("iSeries",inplace=True)
    overview.sort_index(inplace=True)

    overview.to_csv(os.path.join(directory,"overview.csv"))

In [5]:
# pd.read_excel('/Volumes/physio/team/slakrupnik/project/experiments/Sandra/2019_09_25/info.xlsx')

# If you want to parse a single record

In [6]:
# pathToRecord = '/Volumes/physio/team/slakrupnik/project/experiments/MB/Cytokines/cytokines-2.lif'
# pathToRecord = '/Volumes/physio/team/slakrupnik/project/experiments/Sandra/NIKON/2019_08_06/256x32_image_slice2_0la_2Ca.nd2'
pathToRecord = "/Volumes/physio/team/slakrupnik/project/experiments/Ya-Chi/Nov 6 2019.lif"

In [7]:
md = bf.get_omexml_metadata(pathToRecord)
xml = bf.OMEXML(md)
Nimages = xml.get_image_count()
rdr = bf.ImageReader(pathToRecord, perform_init=True)
pathToDir, filename = os.path.split(pathToRecord)
extension = "."+filename.split(".")[-1]
newdir = os.path.join(pathToDir,filename.rstrip(extension))

In [8]:
if not os.path.isdir(newdir):
    os.makedirs(newdir)

In [9]:
OrderedDict([(i,xml.image(i).Name) for i in range(Nimages)])

OrderedDict([(0, 'Image015'),
             (1, 'Image016'),
             (2, 'Image020'),
             (3, 'Image021'),
             (4, 'Image022'),
             (5, 'Image028'),
             (6, 'Image031'),
             (7, 'Series038_6mM glucose'),
             (8, 'Image051'),
             (9, 'Series056_6mM glucose'),
             (10, 'Image060'),
             (11, 'Series062_8mM glucose'),
             (12, 'Series063_8mM glucose'),
             (13, 'Image080'),
             (14, 'Series094'),
             (15, 'Image095'),
             (16, 'Series106_6mM glucose'),
             (17, 'Image112'),
             (18, 'Series113_8mM glucose')])

In [12]:
errd = 0
for iSeries in range(Nimages):
    errd += do_single_Series(iSeries)

if errd==0:
    movecarefully(pathToRecord,os.path.join(newdir,filename+"_done"))

if Nimages>1:
    prepareOverview(newdir)

/Users/srdjan/anaconda2/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: something was off doing 0 in file /Volumes/physio/team/slakrupnik/project/experiments/Ya-Chi/Nov 6 2019.lif.
I'll continue, but with a note in the folder.
/Users/srdjan/anaconda2/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: something was off doing 1 in file /Volumes/physio/team/slakrupnik/project/experiments/Ya-Chi/Nov 6 2019.lif.
I'll continue, but with a note in the folder.
/Users/srdjan/anaconda2/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: something was off doing 4 in file /Volumes/physio/team/slakrupnik/project/experiments/Ya-Chi/Nov 6 2019.lif.
I'll continue, but with a note in the folder.
/Users/srdjan/anaconda2/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: something was off doing 5 in file /Volumes/physio/team/slakrupnik/project/experiments/Ya-Chi/Nov 6 2019.lif.
I'll continue,

# If you want to parse all records within a directory

In [18]:
pathToFolder = '/Volumes/physio/team/slakrupnik/project/experiments/Sandra/NIKON/2019_08_06'

In [19]:
for filename in os.listdir(pathToFolder):
    if "slice3" not in filename: continue
    print (filename)

256x32_image_slice3_0la_04Ca.nd2
256x32_image_slice3_0la_04Ca001.nd2
256x32_image_slice3_0la_10Ca.nd2
256x32_image_slice3_0la_2Ca.nd2
256x32_image_slice3_0la_2Ca001.nd2
256x32_image_slice3_0la_2Ca002.nd2
256x36_image_slice3_0la_2Ca.nd2
256x36_image_slice3_0la_2Ca001.nd2
256x36_image_slice3_0la_2Ca002.nd2
256x36_image_slice3_0la_2Ca003.nd2
512x512_image_slice3_16la_04Ca.nd2
512x512_image_slice3_16la_10Ca.nd2
512X512_image_slice3_16la_2ca.nd2
512x512_image_slice3_16la_2Ca001.nd2
512X512_image_slice3_16la_2ca.txt
512X512_image_slice3_16la_2ca.png
512X512_image_slice3_16la_2ca.npz


In [ ]:
    pathToRecord = os.path.join(pathToFolder,filename)
    try:
        md = bf.get_omexml_metadata(pathToRecord)
    except:
        # if it is something else
        continue
    xml = bf.OMEXML(md)
    Nimages = xml.get_image_count()
    rdr = bf.ImageReader(pathToRecord, perform_init=True)
    pathToDir, filename = os.path.split(pathToRecord)
    extension = "."+filename.split(".")[-1]
    newdir = os.path.join(pathToDir,filename.rstrip(extension))

    if not os.path.isdir(newdir):
        os.makedirs(newdir)

    errd = 0
    for iSeries in range(Nimages):
        errd += do_single_Series(iSeries)

    if errd==0:
        movecarefully(pathToRecord,os.path.join(newdir,filename+"_done"))

    if Nimages>1:
        prepareOverview(newdir)

# Only overview [Leica]

In [15]:
pathToDir = "/Volumes/physio/team/slakrupnik/project/experiments/Sandra/2019_10_15/Experiment38b/"

In [16]:
prepareOverview(pathToDir)

In [17]:
df = pd.read_csv(pathToDir+"overview.csv")

In [18]:
df

,iSeries,Unnamed: 0,T,X,Y,freq,pxSize,pxUnit,filename,Series_Name
0,0,0,1,1024,1024,NaN,0.240352,µm,Experiment38b_0_Image003,Image003.txt
1,1,0,1,1024,1024,NaN,0.258162,µm,Experiment38b_1_Image003,Image003.txt
2,2,0,1,1024,1024,NaN,0.258162,µm,Experiment38b_2_Image010,Image010.txt
3,3,0,1,1024,1024,NaN,0.312712,µm,Experiment38b_3_Image011,Image011.txt
4,4,0,94027,128,64,99.042816,2.518935,µm,Experiment38b_4_Image013,Image013.txt
5,5,0,1,1024,1024,NaN,0.240746,µm,Experiment38b_5_Series014,Series014.txt
6,6,0,1,1024,1024,NaN,0.257745,µm,Experiment38b_6_Image021,Image021.txt
7,7,0,86082,128,64,99.049445,2.076169,µm,Experiment38b_7_Image023,Image023.txt
8,8,0,1,1024,1024,NaN,0.239184,µm,Experiment38b_8_Series024,Series024.txt
9,9,0,1,1024,1024,NaN,0.424412,µm,Experiment38b_9_Image030,Image030.txt


In [10]:
df.query("freq>.6")

,iSeries,Unnamed: 0,T,X,Y,pxSize,pxUnit,filename,Series_Name,freq
10,10,0,4889,512,512,0.531302,µm,Pancreas islet OGB 56_10_Series016,Series016.txt,1.000204
13,13,0,2386,512,512,0.531302,µm,Pancreas islet OGB 56_13_Series020,Series020.txt,1.000419
16,16,0,3658,512,512,0.348258,µm,Pancreas islet OGB 56_16_Series025,Series025.txt,1.000273
19,19,0,1420,512,512,0.348258,µm,Pancreas islet OGB 56_19_Series029,Series029.txt,1.000705
22,22,0,4745,512,512,0.478835,µm,Pancreas islet OGB 56_22_Series033,Series033.txt,1.000210


# Only overview [Nikon]

In [72]:
df = []
for folderName in ["2019_09_25", "2019_08_06"]:
    pathToDir = "/Volumes/physio/team/slakrupnik/project/experiments/Sandra/NIKON/%s/"%folderName
    allFiles = sorted([fn for fn in os.listdir(pathToDir) if fn.endswith(".nd2")])

    for fn in allFiles:
        tentDir = pathToDir+fn.rstrip(".nd2")+"/"
        if os.path.isdir(tentDir):
            tmp = pd.read_csv(tentDir+fn.replace("nd2","txt")).loc[0]
            tmp = tmp.append(pd.Series({"filename":fn.rstrip(".nd2"),"folder":folderName}))
            df += [tmp]


In [73]:
df = pd.DataFrame(df)

In [75]:
df['Tmax'] = df["T"]/df["freq"]

In [77]:
df.sort_values("Tmax", ascending=False)

,T,X,Y,freq,pxSize,pxUnit,filename,folder,Tmax
30,3330,512,512,1.928090,0.415082,µm,512x512_image_slice3_16la_2Ca001,2019_08_06,1727.098039
28,2175,512,512,1.928107,0.415082,µm,512x512_image_slice3_16la_04Ca,2019_08_06,1128.049195
4,99059,256,64,101.066012,1.245245,µm,Slice1_Glu6_TEA_2Ca_Diazoxide_512x64_003,2019_09_25,980.141576
27,1808,512,512,1.928667,0.138107,µm,512x512_image_slice1_16la,2019_08_06,937.435151
29,1430,512,512,1.928000,0.415082,µm,512x512_image_slice3_16la_10Ca,2019_08_06,741.701127
0,64453,256,64,173.340006,1.245245,µm,Slice1_Glu6_Ryn100nM,2019_09_25,371.829918
26,714,512,512,1.928200,0.138107,µm,512x512__image_slice1_16la_10Ca,2019_08_06,370.293511
5,63156,256,64,183.203193,1.660326,µm,Slice2_Glu8_007,2019_09_25,344.731983
7,60151,256,64,206.325159,1.660326,µm,Slice2_Glu8_010,2019_09_25,291.534974
25,481,512,512,1.926945,0.622622,µm,512X512_image_slice3_16la_2ca,2019_08_06,249.617884
